In [ ]:
import os
import pickle
from deepface import DeepFace

# --- Configuration ---
DATABASE_PATH = "database/"
DB_PKL_PATH = "face_database.pkl"
MODEL_NAME = "ArcFace"
DETECTOR_BACKEND = "mtcnn"

print("Configuration loaded.")

In [ ]:

database = {}
print("--- Démarrage de la création de la base de données ---")

for person_name in os.listdir(DATABASE_PATH):
    person_folder_path = os.path.join(DATABASE_PATH, person_name)
    if not os.path.isdir(person_folder_path):
        continue

    database[person_name] = []
    
    print(f"\nTraitement de la personne : {person_name}")
    for image_name in os.listdir(person_folder_path):
        if image_name.lower().endswith(('.png', '.jpg', '.jpeg')):
            image_path = os.path.join(DATABASE_PATH, person_name, image_name)
            try:
                # On utilise .represent() pour obtenir l'embedding
                embedding_obj = DeepFace.represent(
                    img_path=image_path,
                    model_name=MODEL_NAME,
                    detector_backend=DETECTOR_BACKEND,
                    enforce_detection=True
                )
                
                # --- VÉRIFICATION 1 : On regarde ce que DeepFace retourne ---
                if embedding_obj and isinstance(embedding_obj, list) and isinstance(embedding_obj[0], dict) and 'embedding' in embedding_obj[0]:
                    embedding = embedding_obj[0]["embedding"]
                    
                    # --- VÉRIFICATION 2 : On vérifie le type et la taille de l'embedding ---
                    print(f"  ✓ Image '{image_name}' -> Embedding trouvé. Type: {type(embedding)}, Longueur: {len(embedding)}")
                    
                    database[person_name].append(embedding)
                else:
                    print(f"  ✗ Erreur de format inattendue pour l'image '{image_name}'")

            except Exception as e:
                print(f"  ✗ Erreur sur l'image {image_name}: {e}")

final_database = {name: embeddings for name, embeddings in database.items() if embeddings}

with open(DB_PKL_PATH, "wb") as f:
    pickle.dump(final_database, f)

print(f"\n--- SUCCÈS ---")
print(f"Base de données sauvegardée dans '{DB_PKL_PATH}'")

# --- VÉRIFICATION 3 : On inspecte la structure finale de notre base de données ---
print("\n--- Structure de la base de données finale ---")
for name, embeddings_list in final_database.items():
    print(f"  - Personne: {name}")
    print(f"    - Nombre d'embeddings : {len(embeddings_list)}")
    if embeddings_list:
        # On vérifie le premier embedding de la liste
        first_embedding = embeddings_list[0]
        print(f"    - Type du premier embedding : {type(first_embedding)}")
        if isinstance(first_embedding, list):
            print(f"    - Longueur du premier embedding : {len(first_embedding)}")
print("---------------------------------------------")